In [1]:
import json

In [2]:
systems = json.load(open('./data/systems.json'))
data = json.load(open('./graph.json'))

In [3]:
buildings = data['buildings']
graph = data['graph']
edges = [edge for node in graph for edge in node['children']]

In [4]:
output = json.load(open('./our_output.json'))

In [5]:
total_output = {
    "buildings": [],
    "outliers": [],
    "pumps": [],
    "connections": [],
}
for cluster in output["clusters"]:
    total_output["buildings"].extend(cluster['buildings'])
    total_output["outliers"].extend(cluster['buildings'])
    total_output["pumps"].extend(cluster['pumps'])
    total_output["connections"].extend(cluster['connections'])

In [6]:
len(buildings)

34922

In [7]:
# depends on consumption in the close neighbourhood (maybe in cluster)
num_total_pumps = len(total_output["pumps"])
num_heatpumps_s = 0
num_heatpumps_m = 1/4 * num_total_pumps
num_heatpumps_l = 3/4 * num_total_pumps
num_total_pumps

11103

In [8]:
# unused
distrinct_heating_creator = 0
district_heating_connection = 0
heating_element = 0

In [9]:
# depends on consumption of the houses it is connected to
# i can always building big ones...
district_heating_lines = len(total_output["connections"])
average_len = sum([edge['Distance'] for edge in edges])/len(edges)
num_district_heating_line_s = 1/8*district_heating_lines
num_district_heating_line_m = 1/2*district_heating_lines
num_district_heating_line_l = 3/8*district_heating_lines

district_heating_line_s = num_district_heating_line_s*average_len
district_heating_line_m = num_district_heating_line_m*average_len
district_heating_line_l = num_district_heating_line_l*average_len

# TODO: lines loose heat when they are longer

In [10]:
oil_boiler = len(total_output['connections']) # outliers (for now), ggf. photovoltaic
solar_thermal = 0 # TODO: Where does it make sense to add solar thermal where green areas are close by?

In [11]:
transformer = len([building for building in buildings if building['type'] == 'industrial']) # buildings + outliers that are of type 'industrial'
house_connection = len(buildings) - transformer # rest of the buildings + outliers

In [12]:
buildings[0]['summer_consumption']

{'min_electro': 0.6211081327791695,
 'max_electro': 2.735186015458376,
 'avg_electro': 1.7849042776731052,
 'min_heating': 1.4970747780196045,
 'max_heating': 2.1692941915686244,
 'avg_heating': 1.77463397180194,
 'consumption': {'Electro': [0.7025594278931209,
   0.6469026695707675,
   0.6211081327791695,
   0.622077256040198,
   0.64014950457892,
   0.8098171927105894,
   1.40078649697082,
   2.014630957794012,
   2.153712680294308,
   2.1680452007709388,
   2.147783279667381,
   2.270327214691459,
   2.4168153664233447,
   2.4229830116730215,
   2.1088242537383466,
   1.892344853956275,
   1.7998320833691437,
   1.8420826852139744,
   2.3757466371361473,
   2.735186015458376,
   2.7211531066177783,
   2.5276747146305953,
   2.215442018985507,
   1.5817179031903346],
  'Heating': [1.5069381688019838,
   1.5087664061049175,
   1.5092881962541194,
   1.5261084696659908,
   1.5632877593302532,
   1.6432912463162597,
   1.7655880328624702,
   1.8337151879920057,
   1.9514229296059173,
  

In [13]:
buildings[0]['summer_consumption']['consumption']['Electro']

[0.7025594278931209,
 0.6469026695707675,
 0.6211081327791695,
 0.622077256040198,
 0.64014950457892,
 0.8098171927105894,
 1.40078649697082,
 2.014630957794012,
 2.153712680294308,
 2.1680452007709388,
 2.147783279667381,
 2.270327214691459,
 2.4168153664233447,
 2.4229830116730215,
 2.1088242537383466,
 1.892344853956275,
 1.7998320833691437,
 1.8420826852139744,
 2.3757466371361473,
 2.735186015458376,
 2.7211531066177783,
 2.5276747146305953,
 2.215442018985507,
 1.5817179031903346]

In [14]:
for building in buildings:
    if not building['summer_consumption']['consumption']['Heating']:
        building['summer_consumption']['consumption']['Heating'] = []
    if not building['winter_consumption']['consumption']['Heating']:
        building['winter_consumption']['consumption']['Heating'] = []

In [15]:
photovoltaic = 0 # how much consumption do we need and how many areas do we have (-solar_thermal)?
total_building_consumption = sum([(sum(building['summer_consumption']['consumption']['Electro']) if isinstance(building['summer_consumption']['consumption']['Electro'], list) else building['summer_consumption']['consumption']['Electro']) + 
                         (sum(building['winter_consumption']['consumption']['Electro']) if isinstance(building['winter_consumption']['consumption']['Electro'], list) else building['winter_consumption']['consumption']['Electro']) 
                         for building in buildings]) / 48
total_heating_consumption = (sum([(sum(building['summer_consumption']['consumption']['Heating']) if isinstance(building['summer_consumption']['consumption']['Heating'], list) else building['summer_consumption']['consumption']['Heating']) + 
                         (sum(building['winter_consumption']['consumption']['Heating']) if isinstance(building['winter_consumption']['consumption']['Heating'], list) else building['winter_consumption']['consumption']['Heating']) 
                         for building in buildings]) * 1.05 / 3) / 48
total_consumption = total_building_consumption + total_heating_consumption
electric_powerplant = total_consumption/10000 - photovoltaic # total_consumption/10000 - photovoltaic (one powerplant produces 10000 unit of energy)
electric_powerplant

26.689577306977935

In [16]:
# we only need to do storage when we start doing photovoltaic/solarthermal
electricity_storage = 0 # we need to store the difference between the average and the maximum consumption
heat_storage = 0 # we need to store the difference between the average and the maximum consumption

In [17]:
total_invest = heat_storage * systems['storages']['HeatStorage']['invest'] + \
    electricity_storage * systems['storages']['ElectricityStorage']['invest'] + \
    electric_powerplant * systems['supplier']['ElectricPowerPlant']['invest'] + \
    photovoltaic * systems['supplier']['Photovoltaics']['invest'] + \
    house_connection * systems['supplier']['HouseConnection']['invest'] + \
    transformer * systems['supplier']['Transformer']['invest'] + \
    oil_boiler * systems['supplier']['OilBoiler']['invest'] + \
    num_heatpumps_s * systems['supplier']['Heatpump AirWater (small)']['invest'] + \
    num_heatpumps_m * systems['supplier']['Heatpump AirWater (medium)']['invest'] + \
    num_heatpumps_l * systems['supplier']['Heatpump AirWater (big)']['invest'] + \
    district_heating_line_s * systems['line']['DistrictHeatingLine (small)']['investPerM'] + \
    district_heating_line_m * systems['line']['DistrictHeatingLine (medium)']['investPerM'] + \
    district_heating_line_l * systems['line']['DistrictHeatingLine (big)']['investPerM']
total_invest

808887466.8778803

In [18]:
total_operating = heat_storage * systems['storages']['HeatStorage']['operatingCost'] + \
    electricity_storage * systems['storages']['ElectricityStorage']['operatingCost'] + \
    electric_powerplant * systems['supplier']['ElectricPowerPlant']['operatingCost'] + \
    photovoltaic * systems['supplier']['Photovoltaics']['operatingCost'] + \
    house_connection * systems['supplier']['HouseConnection']['operatingCost'] + \
    transformer * systems['supplier']['Transformer']['operatingCost'] + \
    oil_boiler * systems['supplier']['OilBoiler']['operatingCost'] + \
    num_heatpumps_s * systems['supplier']['Heatpump AirWater (small)']['operatingCost'] + \
    num_heatpumps_m * systems['supplier']['Heatpump AirWater (medium)']['operatingCost'] + \
    num_heatpumps_l * systems['supplier']['Heatpump AirWater (big)']['operatingCost'] + \
    num_district_heating_line_s * systems['line']['DistrictHeatingLine (small)']['operatingCost'] + \
    num_district_heating_line_m * systems['line']['DistrictHeatingLine (medium)']['operatingCost'] + \
    num_district_heating_line_l * systems['line']['DistrictHeatingLine (big)']['operatingCost']
total_operating

5050.876518242

In [19]:
total_operating * 24 * 365.25 * 25

1106899588.9727345

In [20]:
total_co2 = heat_storage * systems['storages']['HeatStorage']['co2'] + \
    electricity_storage * systems['storages']['ElectricityStorage']['co2'] + \
    electric_powerplant * systems['supplier']['ElectricPowerPlant']['co2'] + \
    photovoltaic * systems['supplier']['Photovoltaics']['co2'] + \
    house_connection * systems['supplier']['HouseConnection']['co2'] + \
    transformer * systems['supplier']['Transformer']['co2'] + \
    oil_boiler * systems['supplier']['OilBoiler']['co2'] + \
    num_heatpumps_s * systems['supplier']['Heatpump AirWater (small)']['co2'] + \
    num_heatpumps_m * systems['supplier']['Heatpump AirWater (medium)']['co2'] + \
    num_heatpumps_l * systems['supplier']['Heatpump AirWater (big)']['co2'] + \
    num_district_heating_line_s * systems['line']['DistrictHeatingLine (small)']['co2'] + \
    num_district_heating_line_m * systems['line']['DistrictHeatingLine (medium)']['co2'] + \
    num_district_heating_line_l * systems['line']['DistrictHeatingLine (big)']['co2']
total_co2

10089.475830922791

In [21]:
total_co2 * 24 * 365.25 * 25

2211108628.3467298